In [8]:
# --- SECTION 1: IMPORTS & DATA ---
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Load the dataset
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Prepare dropdown options
options = [{'label': 'All Sites', 'value': 'ALL'}]
for site in spacex_df['Launch Site'].unique():
    options.append({'label': site, 'value': site})

# --- SECTION 2: APP INITIALIZATION ---
app = dash.Dash(__name__)

# --- SECTION 3: APP LAYOUT ---
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # TASK 1: Dropdown for Launch Site selection
    dcc.Dropdown(id='site-dropdown',
                 options=options,
                 value='ALL',
                 placeholder="Select a Launch Site here",
                 searchable=True
                ),
    html.Br(),

    # TASK 2: Pie chart for success counts
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),

    # TASK 3: RangeSlider for payload filtering
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                    value=[min_payload, max_payload]),

    # TASK 4: Scatter chart for payload vs. success correlation
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# --- SECTION 4: CALLBACKS ---

# Callback for Pie Chart
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class',
                     names='Launch Site',
                     title='Total Success Launches By Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        # Calculate success (1) vs failure (0) for the specific site
        site_counts = filtered_df['class'].value_counts().reset_index()
        site_counts.columns = ['class', 'count']
        fig = px.pie(site_counts, values='count',
                     names='class',
                     title=f'Total Success Launches for site {entered_site}')
        return fig

# Callback for Scatter Chart
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id="payload-slider", component_property="value")]
)
def update_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    filtered_df = spacex_df[mask]

    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x="Payload Mass (kg)", y="class",
                         color="Booster Version Category",
                         title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        site_specific_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(site_specific_df, x="Payload Mass (kg)", y="class",
                         color="Booster Version Category",
                         title=f'Correlation between Payload and Success for site {entered_site}')
        return fig

from google.colab import output

if __name__ == '__main__':
    # This creates a viewable window inside your Colab cell
    output.serve_kernel_port_as_iframe(port=8050)

    # Run the app
    app.run(host='localhost', port=8050)

<IPython.core.display.Javascript object>

Dash is running on http://localhost:8050/



INFO:dash.dash:Dash is running on http://localhost:8050/



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:8050
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Jan/2026 04:01:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jan/2026 04:01:15] "GET /_dash-component-suites/dash/deps/polyfill@7.v3_4_0m1769831550.12.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jan/2026 04:01:15] "GET /_dash-component-suites/dash/deps/react-dom@18.v3_4_0m1769831550.3.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jan/2026 04:01:15] "GET /_dash-component-suites/dash/deps/prop-types@15.v3_4_0m1769831550.8.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jan/2026 04:01:15] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v3_4_0m1769831550.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jan/2026 04:01:15] "GET /_dash-component-suites/dash/deps/react@18.v3_4_0m1769831550.3